In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

import itertools
import tensorflow as tf
import pandas as pd
import numpy as np
from datetime import date
import os
import yaml
import time

import pandas as pd
import argparse
from tqdm.notebook import tqdm
tqdm.pandas()
%matplotlib inline
import plotly.express as px
from sklearn.metrics import mean_squared_error
from pmdarima.arima import auto_arima
from functools import reduce
from matplotlib import pyplot as plt

from statsmodels.tsa.holtwinters import SimpleExpSmoothing   
from statsmodels.tsa.holtwinters import ExponentialSmoothing
### Create the Stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,GRU,Dropout
from sklearn.preprocessing import MinMaxScaler
from pmdarima.arima import auto_arima
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
import datetime
from dateutil.relativedelta import relativedelta
import holidays
from collections import defaultdict
import statsmodels.api as sm
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

from helper_mp import find_len,state_length_check,data_transform_to_id_level,\
dates_checker,dates_checker_2,padder,mean_absolute_percentage_error,create_dataset,create_lstm,\
fit_model,create_and_train_data_for_lstm,create_and_train_data_for_hw,create_and_train_data_for_arimax,\
create_and_train_data_for_xgboost,compare_models,choose_model_and_forecast,read_params,get_data,\
three_months_mape_calc,weighted_mape_calc,efd_feature_generator,merge_EFD_feature_file,\
create_and_train_data_for_lstm_multivariate,records_maintainer,metric_file_generator,\
create_and_train_data_for_hwwt,create_and_train_data_for_hwws,merge_cardcounts_feature_file,\
metric_file_generator_for_exception_accounts,merge_CMD_feature_file,twelve_months_mape_calc,twelve_months_rmse_calc,mean_calc

In [3]:
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor
import gc
from joblib import Parallel, delayed


## Data Loading

In [5]:
mdm_file_path,raw_data_ten_years,feature_file_path,revenue_date_EFD_by_day_path,daily_gallons_full_path,window_size_lstm,optimizer,loss,models,granularity_level,seasonal_period,trend,seasonal_hw,information_criterion,seasonal_arimax,n_jobs,current_month,validation_window_length,future_prediction_months,date_year,date_month,forecast_target_column,performance_assessment_window,lstm_train_window_length_EFD,num_units_EFD,activation_function_EFD,optimizer_EFD,loss_function_EFD,batch_size_EFD,num_epochs_EFD,MODELS_EFD,ma_mapping_EFD,external_feature_list,learning_rate_xgb,n_estimators_xgb,subsample_xgb,max_depth_xgb,colsample_bytree_xgb,min_child_weight_xgb,train_period_lstm_model,pred_period_lstm_model,num_units_lstm_model,optimizer_lstm_model,loss_function_lstm_model,batch_size_lstm_model,num_epochs_lstm_model,alpha,beta,gamma,card_counts_full_path,model_for_exception_accounts,choice,business_program_name= get_data(config_path="params17.yaml")

In [6]:
mdm_file_path,raw_data_ten_years,feature_file_path,revenue_date_EFD_by_day_path,daily_gallons_full_path,card_counts_full_path,window_size_lstm,optimizer,loss,models,granularity_level,seasonal_period,trend,seasonal_hw,information_criterion,seasonal_arimax,n_jobs,current_month,validation_window_length,future_prediction_months,date_year,date_month,forecast_target_column,performance_assessment_window,lstm_train_window_length_EFD,num_units_EFD,activation_function_EFD,optimizer_EFD,loss_function_EFD,batch_size_EFD,num_epochs_EFD,MODELS_EFD,ma_mapping_EFD,external_feature_list,learning_rate_xgb,n_estimators_xgb,subsample_xgb,max_depth_xgb,colsample_bytree_xgb,min_child_weight_xgb,train_period_lstm_model,pred_period_lstm_model,num_units_lstm_model,optimizer_lstm_model,loss_function_lstm_model,batch_size_lstm_model,num_epochs_lstm_model,alpha,beta,gamma,model_for_exception_accounts,choice,business_program_name

('D:/Users/W505723/Downloads/Sanitized_dataset/NewSanitisedData/mdm_final_table_sanitized.csv',
 'D:/Users/W505723/Downloads/Sanitized_dataset/EDW_{}_volume_Sanitized.csv',
 'D:/Users/W505723/WEX_Data_exploration/EDA_NewSanitisedData/grouped_efd_per_month.csv',
 'D:/Users/W505723/WEX_Data_exploration/EDA_NewSanitisedData/mv_features/revenue_date_EFD_by_day.txt',
 'D:/Users/W505723/WEX_Data_exploration/EDA_NewSanitisedData/mv_features/daily_gallons_full.csv',
 'D:/Users/W505723/WEX_Data_exploration/EDA_NewSanitisedData/mv_features/card_counts_sanitized.csv',
 36,
 'adam',
 'mean_squared_error',
 ['HW', 'ARIMAX', 'XGB', 'LSTM_Multivariate'],
 'partyparentid',
 12,
 'mul',
 'mul',
 'aic',
 True,
 -1,
 datetime.date(2018, 12, 1),
 12,
 24,
 'revenue_year',
 'revenue_month',
 'puchase_gallons_qty',
 12,
 24,
 16,
 'relu',
 'adam',
 'mean_squared_error',
 16,
 100,
 ['efd', 'efd_ma3m', 'efd_ma6m'],
 {'efd_ma3m': 91, 'efd_ma6m': 182},
 ['lag_12',
  'lag_9',
  'lag_6',
  'lag_3',
  'lag_2',
  

In [7]:
#loading ten years raw data
dataset_pd = pd.DataFrame()
for i in tqdm(range(2010,2023,1)):
    df = pd.read_csv(raw_data_ten_years.format(i))
    df = df.dropna(subset=['customer_account_id'])
    dataset_pd = pd.concat([dataset_pd, df], ignore_index=True)

## data pre processing
dataset_pd['customer_account_id'] = dataset_pd['customer_account_id'].astype('int64')
dataset_pd['revenue_date'] = pd.to_datetime(dataset_pd['revenue_date'])
dataset_pd.sort_values(['revenue_date'], inplace=True)

#loading mdm data
mdm = pd.read_csv(mdm_file_path)

  0%|          | 0/13 [00:00<?, ?it/s]

In [8]:
dataset_pd = dataset_pd[dataset_pd['customer_business_program_name']==business_program_name]


mdm = mdm[mdm['businessprogramname']==business_program_name]


In [9]:
dataset_pd[forecast_target_column] = np.where(dataset_pd[forecast_target_column] < 0, 0, dataset_pd[forecast_target_column])

dataset_pd.shape,mdm.shape

((3166338, 13), (82374, 29))

## EFD Prediction

In [10]:
## Main holidays
holidays_me = pd.DataFrame()
for yr in range(2008, 2025, 1):
    me_cal = pd.DataFrame(holidays.US(state='ME', years=yr).items(), columns = ['REV_CALENDAR_DATE', 'HOLIDAY_NAME'])
    me_cal['REV_CALENDAR_DATE'] = pd.to_datetime(me_cal['REV_CALENDAR_DATE'])
    me_cal['HOLIDAY'] = 1
    holidays_me = pd.concat([holidays_me, me_cal])

In [16]:
efd_features_monthly = efd_feature_generator(holidays_me,str(current_month),lstm_train_window_length_EFD,num_units_EFD,activation_function_EFD,optimizer_EFD,loss_function_EFD,batch_size_EFD,num_epochs_EFD,MODELS_EFD,ma_mapping_EFD,revenue_date_EFD_by_day_path,daily_gallons_full_path,validation_window_length,future_prediction_months)

2016-01-01 00:00:00 2017-12-31 00:00:00 2018-01-01 00:00:00 2018-12-31 00:00:00 2019-01-01 00:00:00 2019-12-31 00:00:00
371
371
371


In [21]:
efd_features_monthly.tail()

,rev_date,REV_EQUIVALENT_FUEL_DAY_FACTOR_efd,REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma3m,REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma6m
109,2019-08-01,24.317775,23.541731,23.154284
110,2019-09-01,22.735426,22.539486,22.188164
111,2019-10-01,25.260967,23.850849,23.470979
112,2019-11-01,22.434596,21.292680,20.994602
113,2019-12-01,22.589182,22.916341,22.306160


In [18]:
efd_features_monthly=efd_features_monthly.reset_index()

In [20]:
efd_features_monthly = efd_features_monthly.rename(columns = {'index':'rev_date'})



In [22]:
efd_features_monthly['rev_date'] = pd.to_datetime(efd_features_monthly['rev_date'])

In [23]:
efd_features_monthly.tail()

,rev_date,REV_EQUIVALENT_FUEL_DAY_FACTOR_efd,REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma3m,REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma6m
109,2019-08-01,24.317775,23.541731,23.154284
110,2019-09-01,22.735426,22.539486,22.188164
111,2019-10-01,25.260967,23.850849,23.470979
112,2019-11-01,22.434596,21.292680,20.994602
113,2019-12-01,22.589182,22.916341,22.306160


## Google mobility feature for covid months

In [24]:

def generate_mobility_feature(current_month, future_prediction_months,
                              start_date = '2010-07-01', 
                              state_name = None,
                              mobility_col = 'retail_and_recreation_percent_change_from_baseline',
                              pre_covid_baseline = 0,
                              post_covid_baseline = -3
                             ):
    '''
        Function to generate mobility feature using Google mobility data for the US
        state_name: Add complete name of state, eg. 'California', to get state level mobility feature, 
                    else None for national level mobility feature
        mobility_col: Select the mobility type (needs to be the exact column name from the file)
        pre-covid baseline: This value will be reflected for all pre-covid months
        post-covid baseline: This value will be reflected for all forecasting window months
    '''
    import zipfile
    from urllib.request import urlopen
    import shutil
    import os
    from dateutil.relativedelta import relativedelta

    url = "https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip"
    file_name = 'Region_Mobility_Report_CSVs.zip'

    # extracting zipfile from URL
    with urlopen(url) as response, open(file_name, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)

        # extracting required file from zipfile
        with zipfile.ZipFile(file_name) as zf:
            zf.extract('2020_US_Region_Mobility_Report.csv')
            zf.extract('2021_US_Region_Mobility_Report.csv')
            zf.extract('2022_US_Region_Mobility_Report.csv')

    # deleting the zipfile from the directory
    os.remove('Region_Mobility_Report_CSVs.zip')

    # loading data from the files
    mobility = pd.read_csv('2020_US_Region_Mobility_Report.csv').append(
                pd.read_csv('2021_US_Region_Mobility_Report.csv')).append(
                pd.read_csv('2022_US_Region_Mobility_Report.csv')).rename(
                columns = {'sub_region_1':'state'})

    if state_name == None:
        mobility = mobility[(mobility.country_region_code == 'US') & (mobility.state.isnull()) & (mobility.sub_region_2.isnull())][[
                'date', mobility_col
            ]]
    else:
        mobility = mobility[(mobility.country_region_code == 'US') & (mobility.state==state_name) & (mobility.sub_region_2.isnull())][[
                'date', mobility_col
            ]]

    mobility = mobility.sort_values('date')

    mobility['date'] = pd.to_datetime(mobility.date, format='%Y-%m-%d')
    mobility['month'] = mobility['date'].to_numpy().astype('datetime64[M]')
    mobility = mobility.groupby(['month']).mean().reset_index()

    pre_covid_end_month = mobility.month.min() - relativedelta(months=1)
    post_covid_start_month = mobility.month.max() + relativedelta(months=1)
    future_time_series_start_month = pd.to_datetime(current_month, format = '%Y-%m-%d') + relativedelta(months=1)
    end_month = pd.to_datetime(current_month, format = '%Y-%m-%d') + relativedelta(months=future_prediction_months)

    pre_covid_date_range = pd.date_range(start=start_date, end=pre_covid_end_month, freq='MS')
    post_covid_date_range = pd.date_range(start=min(post_covid_start_month, future_time_series_start_month), 
                                          end=end_month, freq='MS')
    pre_covid_feat = pd.DataFrame({'month': pre_covid_date_range, 
                                   mobility_col:[pre_covid_baseline]*len(pre_covid_date_range)})
    post_covid_feat = pd.DataFrame({'month': post_covid_date_range, 
                                    mobility_col:[post_covid_baseline]*len(post_covid_date_range)})
    mobility = mobility[mobility.month < future_time_series_start_month].append(
        pre_covid_feat).append(post_covid_feat).sort_values('month').reset_index(drop=True)
    
    return mobility

In [25]:
covid_mobility_df = generate_mobility_feature(current_month = str(current_month), future_prediction_months = validation_window_length)

covid_mobility_df = covid_mobility_df.rename(columns={'month':'rev_date'})
covid_mobility_df = covid_mobility_df.groupby('rev_date',as_index=False).agg(retail_and_recreation_percent_change_from_baseline = ('retail_and_recreation_percent_change_from_baseline',sum))
covid_mobility_df = covid_mobility_df[covid_mobility_df['rev_date']<=current_month+ pd.DateOffset(months = validation_window_length)]
covid_mobility_df.tail()

,rev_date,retail_and_recreation_percent_change_from_baseline
109,2019-08-01,-3.0
110,2019-09-01,-3.0
111,2019-10-01,-3.0
112,2019-11-01,-3.0
113,2019-12-01,-3.0


## Pipeline

In [26]:
%timeit
def processing_engine():    
    start_time = time.time()
    # Import and transform data to the desired level (WEX_ID-month level or business_program_name-month level)
    grouped_id_level = data_transform_to_id_level(mdm,dataset_pd,date_year,date_month,forecast_target_column,granularity_level)

    # Account should have atleast 2 years of data to train check
    if choice == 1: #For only Forecast
        print(1)
        grouped_id_level['year_check']  = grouped_id_level.progress_apply(dates_checker_2,current_month=current_month,axis=1)
        grouped_id_level = grouped_id_level[grouped_id_level['year_check']==1]
#         grouped_id_level=grouped_id_level.head(30)
    
    if choice == 2: #For Forecast and Compare
        print(2)
        grouped_id_level['year_check']  = grouped_id_level.progress_apply(dates_checker,current_month=current_month,axis=1)
        grouped_id_level = grouped_id_level[grouped_id_level['year_check']==1]
        grouped_id_level=grouped_id_level.head(50)

    # Missing Value treatment
    combined_date_list = []
    combined_gallons_list = []
    ans = grouped_id_level.progress_apply(padder,
                                          combined_date_list=combined_date_list,
                                          combined_gallons_list=combined_gallons_list,
                                          validation_window_length=validation_window_length,
                                          current_month=current_month,axis=1) 
    grouped_id_level['rev_date'] = combined_date_list
    grouped_id_level['gallons_list'] = combined_gallons_list
    grouped_id_level['total_records_after_padding'] = grouped_id_level.progress_apply(find_len,axis=1)
    grouped_id_level_ind = grouped_id_level.reset_index()
    grouped_id_level_ind['state'] = grouped_id_level_ind.progress_apply(state_length_check,axis=1)

    
    # Adding EFD as a feature
    grouped_id_level_ind['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd'],\
    grouped_id_level_ind['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma3m'],grouped_id_level_ind['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma6m']= zip(*grouped_id_level_ind.progress_apply(merge_EFD_feature_file,efd_features_monthly=efd_features_monthly,axis=1))
    
    # Adding covid mobility as a feature
    grouped_id_level_ind['covid_mobility_feature']=grouped_id_level_ind.progress_apply(merge_CMD_feature_file,
                                                                                       cmd_features_monthly=covid_mobility_df,
                                                                                       axis=1)
    
    
    print("--- %s seconds for data preparation---" % (time.time() - start_time))
    # Building models from the multiple options available 
    start_time = time.time()
    
    models_trained = []     

#     if 'HW' in models:
#         print('Running HW')
#         rows = [(grouped_id_level_ind.loc[i],
#                  current_month,
#                  validation_window_length,
#                  alpha,
#                  beta,
#                  gamma,
#                  granularity_level) for i in grouped_id_level_ind.index]
#         results_hw_list = Parallel(n_jobs=mp.cpu_count())(delayed(create_and_train_data_for_hw)(a,b,c,d,e,f,g) for a,b,c,d,e,f,g in rows)

#         results_hw = pd.DataFrame(results_hw_list, columns=[granularity_level, 'mape_hw', 'rmse_hw',\
#                             'agorithm_used_hw', 'rev_date_hw', 'ground_truth_hw', 'predictions_hw'])
#         results_hw['agorithm_used_hw'] = 'HW'
#         models_trained.append(results_hw)
#         del rows
#         gc.collect()

    if 'XGB' in models and len(external_feature_list)>0: 
        start_time_xgb = time.time()
        print('Running XGB')
        rows = [(grouped_id_level_ind.loc[i],
                current_month,
                validation_window_length,
                external_feature_list,
                learning_rate_xgb,
                n_estimators_xgb,
                subsample_xgb,
                max_depth_xgb,
                colsample_bytree_xgb,
                min_child_weight_xgb,
                granularity_level) for i in grouped_id_level_ind.index]
        results_xgb_list = Parallel(n_jobs=mp.cpu_count())(delayed(create_and_train_data_for_xgboost)(a,b,c,d,e,f,g,h,i,j,k) for a,b,c,d,e,f,g,h,i,j,k in rows)

        results_xgb = pd.DataFrame(results_xgb_list, columns=[granularity_level, 'mape_xgb', 'rmse_xgb', 
                            'agorithm_used_xgb', 'rev_date_xgb', 'ground_truth_xgb', 'predictions_xgb'])
        results_xgb['agorithm_used_xgb'] = 'XGB'
        models_trained.append(results_xgb)
        del rows
        gc.collect()  
        print("--- %s seconds for running XGB---" % (time.time() - start_time_xgb))
#     if 'ARIMAX' in models and len(external_feature_list)>0:
#         print('Running ARIMAX')
#         rows = [(grouped_id_level_ind.loc[i], 
#                  current_month,
#                  validation_window_length,
#                  information_criterion,
#                  seasonal_arimax,
#                  external_feature_list,
#                  n_jobs,
#                  granularity_level) for i in grouped_id_level_ind.index]
#         results_arimax_list = Parallel(n_jobs=mp.cpu_count())(delayed(create_and_train_data_for_arimax)(a,b,c,d,e,f,g,h) for a,b,c,d,e,f,g,h in rows)
#         results_arimax = pd.DataFrame(results_arimax_list, columns=[
#             granularity_level, 'mape_arimax', 'rmse_arimax', 'agorithm_used_arimax',
#             'rev_date_arimax', 'ground_truth_arimax', 'predictions_arimax'
#         ])
#         results_arimax['agorithm_used_arimax'] = 'ARIMAX'
#         models_trained.append(results_arimax)
#         del rows
#         gc.collect()


#     if 'LSTM_Multivariate' in models and len(external_feature_list)>0:
#         print('Running LSTM_Multivariate')
#         rows = [(grouped_id_level_ind.loc[i],
#                  current_month,
#                  validation_window_length,
#                  external_feature_list,
#                  train_period_lstm_model,
#                  pred_period_lstm_model,
#                  optimizer_lstm_model,
#                  loss_function_lstm_model,
#                  batch_size_lstm_model,
#                  num_epochs_lstm_model,
#                 granularity_level) for i in grouped_id_level_ind.index]
#         results_lstm_list = Parallel(n_jobs=mp.cpu_count())(delayed(create_and_train_data_for_lstm_multivariate)(a,b,c,d,e,f,g,h,i,j,k) for a,b,c,d,e,f,g,h,i,j,k in rows)
        
#         results_lstm = pd.DataFrame(results_lstm_list, columns=[
#             granularity_level, 'mape_lstm_multivariate', 'rmse_lstm_multivariate', 
#             'agorithm_used_lstm_multivariate',
#             'rev_date_lstm_multivariate', 'ground_truth_lstm_multivariate', 
#             'predictions_lstm_multivariate'
#         ])
#         results_lstm['agorithm_used_lstm_multivariate'] = 'LSTM_Multivariate'
#         models_trained.append(results_lstm)
#         del rows
#         gc.collect()
 

    print("--- %s seconds for building models---" % (time.time() - start_time))
        
    ## COMPARE MODELS
    print('Initialising Compare models')
    start_time = time.time()
    id_best_algo_df = compare_models(models_trained,granularity_level)
    grouped_id_level_ind = grouped_id_level_ind.merge(id_best_algo_df,on=granularity_level,how='left')
    print("--- %s seconds for comparing models---" % (time.time() - start_time))
    
    ## CHOOSE and FORECAST using the best model
    print('Initialising Forecasting ')
    start_time = time.time()
    
    rows = [(grouped_id_level_ind.loc[i],
            current_month,
            validation_window_length,
            information_criterion,
            seasonal_arimax,
            external_feature_list,
            n_jobs,
            learning_rate_xgb,
            n_estimators_xgb,
            subsample_xgb,
            max_depth_xgb,
            colsample_bytree_xgb,
            min_child_weight_xgb,
            train_period_lstm_model,
            pred_period_lstm_model,
            optimizer_lstm_model,
            loss_function_lstm_model,
            batch_size_lstm_model,
            num_epochs_lstm_model,
            alpha,
            beta,
            gamma,
            granularity_level) for i in grouped_id_level_ind.index]
    final_res_list = Parallel(n_jobs=mp.cpu_count())(delayed(choose_model_and_forecast)(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w) for a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w in rows)
    
    final_res = pd.DataFrame(final_res_list, columns=[granularity_level,
                                                     'overall_mape(performance_assessment_period)',
                                                     'overall_rmse(performance_assessment_period)',
                                                     'model_chosen',
                                                     'month',
                                                     'gallons_actual(performance_assessment_period)',
                                                     'gallons_forecasts(performance_assessment_period)'])
    del rows
    gc.collect()
    error_file = final_res[final_res['model_chosen']=='ERROR']
    final_res = final_res[final_res['model_chosen']!='ERROR']
    
    #final_res['wex_id'],final_res['performance_assesment_mape'],final_res['performance_assesment_rmse'],final_res['agorithm_used'],final_res['performance_assesment_rev_date'],final_res['performance_assesment_ground_truth'],final_res['performance_assesment_forecasts'],final_res['24_months_forecasts']= zip(*grouped_wex_id_level_ind.progress_apply(choose_model_and_forecast,future_prediction_months=future_prediction_months,current_month=current_month,performance_assessment_window=performance_assessment_window,trend=trend,seasonal_hw=seasonal_hw,seasonal_periods=seasonal_period,information_criterion=information_criterion,seasonal_arimax=seasonal_arimax,external_feature_list=external_feature_list,n_jobs=n_jobs,learning_rate_xgb=learning_rate_xgb,n_estimators_xgb=n_estimators_xgb,subsample_xgb=subsample_xgb,max_depth_xgb=max_depth_xgb,colsample_bytree_xgb=colsample_bytree_xgb,min_child_weight_xgb=min_child_weight_xgb,axis=1))
    print("--- %s seconds for forecasting using best model---" % (time.time() - start_time))
    
    ## Output the required files
    start_time = time.time()
    final_res['3_month_mape(performance_assesment_period)'] = final_res.progress_apply(three_months_mape_calc,axis=1)
    final_res['time_weighted_mape(performance_assesment_period)'] = final_res.progress_apply(weighted_mape_calc,axis=1)

    final_res['overall_mape(performance_assessment_period)'] = final_res['overall_mape(performance_assessment_period)'].astype('float64')
    final_res['time_weighted_mape(performance_assesment_period)'] = final_res['time_weighted_mape(performance_assesment_period)'].astype('float64')
    final_res['3_month_mape(performance_assesment_period)'] = final_res['3_month_mape(performance_assesment_period)'].astype('float64')
    
    final_res['overall_mape(performance_assessment_period)'] = round(final_res['overall_mape(performance_assessment_period)'],2)
    final_res['time_weighted_mape(performance_assesment_period)'] = round(final_res['time_weighted_mape(performance_assesment_period)'],2)
    final_res['3_month_mape(performance_assesment_period)'] = round(final_res['3_month_mape(performance_assesment_period)'],2)
    df_dict = dict()
    for idx,dfs in enumerate(models_trained):
        model = dfs.iloc[0,3]
        df_dict[model] = idx
    
    final_res['overall_mape(evaluation_period)'],final_res['3_month_mape(evaluation_period)'],\
    final_res['time_weighted_mape(evaluation_period)'],final_res['monthly_avg_volume(evaluation_period)'] =\
    zip(*final_res.progress_apply(records_maintainer,
                                  df_dict=df_dict,models_trained=models_trained,
                                  granularity_level=granularity_level,axis=1))
    
    metrics_file = final_res[[granularity_level,'model_chosen','monthly_avg_volume(evaluation_period)','overall_mape(evaluation_period)',
                             '3_month_mape(evaluation_period)','time_weighted_mape(evaluation_period)',
                             'overall_mape(performance_assessment_period)','3_month_mape(performance_assesment_period)',
                             'time_weighted_mape(performance_assesment_period)']]
    
    
    final_res = final_res.merge(grouped_id_level_ind[[granularity_level,'rev_date','gallons_list']], on =granularity_level,how='left')   
    metric_file = pd.DataFrame()
    metric_file['data'] = final_res.progress_apply(metric_file_generator,
                                                   df_dict=df_dict, models_trained=models_trained,models=models,
                                                   granularity_level=granularity_level,axis=1) 
    total_forecast_file = pd.DataFrame()
    for i in range(len(metric_file)):
        total_forecast_file = total_forecast_file.append(metric_file.iloc[i][0],ignore_index=True)
        
    ##Exception accounts
    print('Handling Exception accounts')
    final_res_B = final_res[final_res['overall_mape(evaluation_period)']>=40]
    final_res11gt40_wids = list(final_res_B[granularity_level])

    grouped_id_level = data_transform_to_id_level(mdm,dataset_pd,date_year,date_month,forecast_target_column,granularity_level)
    
    if choice == 1:
        print(1)
        grouped_id_level['year_check']  = grouped_id_level.progress_apply(dates_checker_2,current_month=current_month,axis=1)
    if choice == 2:
        print(2)
        grouped_id_level['year_check']  = grouped_id_level.progress_apply(dates_checker,current_month=current_month,axis=1)

    grouped_id_level_ind_A = grouped_id_level.reset_index()

    grouped_id_level_ind_A['ev_mape_gt40_check']  = np.where((grouped_id_level_ind_A[granularity_level].isin(final_res11gt40_wids)),1,0)
    grouped_id_level = grouped_id_level_ind_A.set_index(granularity_level)

    grouped_id_level_AB = grouped_id_level[(grouped_id_level['year_check']==2) | (grouped_id_level['ev_mape_gt40_check']==1)]

    #Padding
    combined_date_list_AB = []
    combined_gallons_list_AB = []
    ans_AB = grouped_id_level_AB.progress_apply(padder,
                                                combined_date_list=combined_date_list_AB,
                                                combined_gallons_list=combined_gallons_list_AB,
                                                validation_window_length=validation_window_length,
                                                current_month=current_month,axis=1) 
    grouped_id_level_AB['rev_date'] = combined_date_list_AB
    grouped_id_level_AB['gallons_list'] = combined_gallons_list_AB
    grouped_id_level_AB['total_records_after_padding'] = grouped_id_level_AB.progress_apply(find_len,axis=1)
    grouped_id_level_ind_AB = grouped_id_level_AB.reset_index()
    grouped_id_level_ind_AB['state'] = grouped_id_level_ind_AB.progress_apply(state_length_check,axis=1)    
    grouped_id_level_ind_AB = grouped_id_level_ind_AB[grouped_id_level_ind_AB['total_records_after_padding']>=18]
    
    ##storing ids
    smallaccounts_ids = list(grouped_id_level_ind_AB[grouped_id_level_ind_AB['year_check']==2][granularity_level])

    grouped_id_level_ind_AB['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd'],\
    grouped_id_level_ind_AB['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma3m'],\
    grouped_id_level_ind_AB['REV_EQUIVALENT_FUEL_DAY_FACTOR_efd_ma6m']=\
    zip(*grouped_id_level_ind_AB.progress_apply(merge_EFD_feature_file,efd_features_monthly=efd_features_monthly,axis=1))

    # Adding covid mobility as a feature
    grouped_id_level_ind_AB['covid_mobility_feature']=grouped_id_level_ind_AB.progress_apply(merge_CMD_feature_file,
                                                                                       cmd_features_monthly=covid_mobility_df,
                                                                                       axis=1)
    
    
    ## Modelling for Exception accounts
    grouped_id_level_ind_AB = grouped_id_level_ind_AB.sample(10)
    #ARIMAX
    grouped_id_level_ind_AB = grouped_id_level_ind_AB.assign(best_algo='ARIMAX')   #model_for_exception_accounts 
    rows = [(grouped_id_level_ind_AB.loc[i],
            current_month,
            validation_window_length,
            information_criterion,
            seasonal_arimax,
            external_feature_list,
            n_jobs,
            learning_rate_xgb,
            n_estimators_xgb,
            subsample_xgb,
            max_depth_xgb,
            colsample_bytree_xgb,
            min_child_weight_xgb,
            train_period_lstm_model,
            pred_period_lstm_model,
            optimizer_lstm_model,
            loss_function_lstm_model,
            batch_size_lstm_model,
            num_epochs_lstm_model,
            alpha,
            beta,
            gamma,
            granularity_level) for i in grouped_id_level_ind_AB.index]
    final_res_exp_list_1 = Parallel(n_jobs=mp.cpu_count())(delayed(choose_model_and_forecast)(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w) for a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w in rows)

    final_res_exp_1 = pd.DataFrame(final_res_exp_list_1, columns=[granularity_level,
                                                     'overall_mape(performance_assessment_period)',
                                                     'overall_rmse(performance_assessment_period)',
                                                     'model_chosen',
                                                     'month',
                                                     'gallons_actual(performance_assessment_period)',
                                                     'gallons_forecasts(performance_assessment_period)'])
    del rows
    gc.collect()   
    
    #LSTM_MULTIVARIATE
    grouped_id_level_ind_AB = grouped_id_level_ind_AB.assign(best_algo='LSTM_MULTIVARIATE')   #model_for_exception_accounts 
    train_period_lstm_model_exp = 5
    rows = [(grouped_id_level_ind_AB.loc[i],
            current_month,
            validation_window_length,
            information_criterion,
            seasonal_arimax,
            external_feature_list,
            n_jobs,
            learning_rate_xgb,
            n_estimators_xgb,
            subsample_xgb,
            max_depth_xgb,
            colsample_bytree_xgb,
            min_child_weight_xgb,
            train_period_lstm_model_exp,
            pred_period_lstm_model,
            optimizer_lstm_model,
            loss_function_lstm_model,
            batch_size_lstm_model,
            num_epochs_lstm_model,
            alpha,
            beta,
            gamma,
            granularity_level) for i in grouped_id_level_ind_AB.index]
    final_res_exp_list_2 = Parallel(n_jobs=mp.cpu_count())(delayed(choose_model_and_forecast)(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w) for a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w in rows)

    final_res_exp_2 = pd.DataFrame(final_res_exp_list_2, columns=[granularity_level,
                                                     'overall_mape(performance_assessment_period)',
                                                     'overall_rmse(performance_assessment_period)',
                                                     'model_chosen',
                                                     'month',
                                                     'gallons_actual(performance_assessment_period)',
                                                     'gallons_forecasts(performance_assessment_period)'])
    del rows
    gc.collect()
    
    #ENSEMBLER
    final_res_exp = pd.DataFrame()
    gallons_forecasts = []

    for i in range(final_res_exp_2.shape[0]):
        gallons_forecasts.append(mean_calc(final_res_exp_1['gallons_forecasts(performance_assessment_period)'].iloc[i],final_res_exp_2['gallons_forecasts(performance_assessment_period)'].iloc[i]))
    final_res_exp[granularity_level] = final_res_exp_2[granularity_level]
    final_res_exp['gallons_actual(performance_assessment_period)'] = final_res_exp_2['gallons_actual(performance_assessment_period)']
    final_res_exp['gallons_forecasts(performance_assessment_period)'] = gallons_forecasts
    final_res_exp['model_chosen'] = 'ENSEMBLE(LSTM_and_ARIMAX)'
    final_res_exp['month'] = final_res_exp_2['month']
    final_res_exp['overall_mape(performance_assessment_period)'] = final_res_exp.progress_apply(twelve_months_mape_calc,axis=1)
    final_res_exp['overall_rmse(performance_assessment_period)'] = final_res_exp.progress_apply(twelve_months_rmse_calc,axis=1)

    error_file_exp = final_res_exp[final_res_exp['model_chosen']=='ERROR']
    final_res_exp = final_res_exp[final_res_exp['model_chosen']!='ERROR']

    final_res_exp['3_month_mape(performance_assesment_period)'] = final_res_exp.progress_apply(three_months_mape_calc,axis=1)
    final_res_exp['time_weighted_mape(performance_assesment_period)'] = final_res_exp.progress_apply(weighted_mape_calc,axis=1)

    final_res_exp = final_res_exp.merge(grouped_id_level_ind_AB[[granularity_level,'rev_date','gallons_list']],on=granularity_level,how='left')
    metrics_fileAB = final_res_exp[[granularity_level,'model_chosen','overall_mape(performance_assessment_period)',
                              '3_month_mape(performance_assesment_period)','time_weighted_mape(performance_assesment_period)']]
    
    metric_fileAB = pd.DataFrame()
    metric_fileAB['data'] = final_res_exp.progress_apply(metric_file_generator_for_exception_accounts,granularity_level=granularity_level,axis=1) 
    total_forecast_fileAB = pd.DataFrame()
    for i in range(len(metric_fileAB)):
        total_forecast_fileAB = total_forecast_fileAB.append(metric_fileAB.iloc[i][0],ignore_index=True)
    
    exception_ids = list(final_res_exp[granularity_level])   
    metrics_file = metrics_file[~(metrics_file[granularity_level].isin(exception_ids))]
    total_forecast_file = total_forecast_file[~(total_forecast_file[granularity_level].isin(exception_ids))]
    metrics_file = metrics_file[~(metrics_file[granularity_level].isin(exception_ids))]
    final_res = final_res[~(final_res[granularity_level].isin(exception_ids))]
    
    total_forecast_file = total_forecast_file.append(total_forecast_fileAB).reset_index(drop=True)
    metrics_file = metrics_file.append(metrics_fileAB).reset_index(drop=True)
    final_res = final_res.append(final_res_exp).reset_index(drop=True)
    
    ## Putting status  flags to differentiate different accounts
    conditions = [final_res[granularity_level].isin(final_res11gt40_wids), 
                  final_res[granularity_level].isin(smallaccounts_ids)]
    choices =    ['accounts with greater than 40 evaluation period MAPE', 
                  'accounts with less than 36 datapoints']
    final_res['status_flag'] = np.select(conditions, choices, default='eligible accounts')
    
    conditions = [total_forecast_file[granularity_level].isin(final_res11gt40_wids), 
                  total_forecast_file[granularity_level].isin(smallaccounts_ids)]
    choices =    ['accounts with greater than 40 evaluation period MAPE', 
                  'accounts with less than 36 datapoints']
    total_forecast_file['status_flag'] = np.select(conditions, choices, default='eligible accounts')
    
    conditions = [metrics_file[granularity_level].isin(final_res11gt40_wids), 
                  metrics_file[granularity_level].isin(smallaccounts_ids)]
    choices =    ['accounts with greater than 40 evaluation period MAPE', 
                  'accounts with less than 36 datapoints']
    metrics_file['status_flag'] = np.select(conditions, choices, default='eligible accounts')
    
    if choice == 1:
        print(1)
        metrics_file.drop(['overall_mape(performance_assessment_period)',
                           '3_month_mape(performance_assesment_period)',
                           'time_weighted_mape(performance_assesment_period)'],axis=1,inplace=True)

    metrics_file.to_csv('metrics_file.csv',index=False)
    total_forecast_file.to_csv('forecasts.csv',index=False)
    final_res.to_csv('final_results.csv',index=False)
    print("--- %s seconds for outputting the required files---" % (time.time() - start_time))


    return models_trained,grouped_id_level_ind,id_best_algo_df,final_res,total_forecast_file,metrics_file,error_file,error_file_exp

In [27]:
import sys
old_stdout = sys.stdout
log_file = open("message.log","w")
sys.stdout = log_file
models_trained,grouped_id_level_ind,id_best_algo_df,final_res,\
total_forecast_file,metrics_file,error_file,error_file_exp= processing_engine()
sys.stdout = old_stdout
log_file.close()

  0%|          | 0/3812 [00:00<?, ?it/s]

  0%|          | 0/3812 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [28]:
final_res['model_chosen'].value_counts()

XGB                          49
ENSEMBLE(LSTM_and_ARIMAX)    10
Name: model_chosen, dtype: int64